i choose this simple problem to pratice Factory Method pattern

In [8]:
from abc import ABC, abstractmethod
from pydantic import BaseModel, field_validator, Field
from rich.prompt import Prompt

In [9]:
# Product

class Multiples(BaseModel, ABC):
    total: int = 0
    limit: int = Field(..., description = "maximum number of processing")
    
    @abstractmethod
    def solve(self) -> int:
        """محاسبه‌ی مجموع مضارب بر اساس روش انتخابی"""
        pass

In [10]:
# Concrete Product

class BruteForce(Multiples):
    def solve(self) -> int:
        total = 0
        for i in range(self.limit):
            if i % 3 == 0 or i % 5 == 0:
                total += i
            
        return total
    
class FormulaMethod(Multiples):
    def solve(self) -> int:
        def sum_divisible_by(n: int) -> int:
            p = (self.limit - 1) // n
            return n * (p * (p + 1)) // 2
        
        return sum_divisible_by(n=3) + sum_divisible_by(n=5) - sum_divisible_by(n=15)


In [12]:
# Concrete Factory

class SolverFactory(BaseModel):
    method: str = "brute"
    limit: int = 10

    @field_validator("method", mode="after")
    def check_input(cls, v: str):
        if v.strip().lower() not in ("brute", "formula"):
            raise ValueError("Enter valid value: brute or formula")
        return v
    
    @abstractmethod
    def create_solver(method: str, limit: int) -> Multiples:
        method = method.lower().strip()
        
        if method == "brute":
            return BruteForce(limit=limit)
        elif method == "formula":
            return FormulaMethod(limit=limit)
        else:
            raise ValueError(f"Unknown method: {method}")

In [15]:
# Client

from pydantic import ValidationError


if __name__ == "__main__":
    method = Prompt.ask(
        prompt="Enter method of solving problem",
        choices=["brute", "formula"],
        default="formula",
        case_sensitive=False,
    )
    
    limit = Prompt.ask(
        prompt="Enter limit of numbers",
        default=10,
    )

    try: 
        solver = SolverFactory.create_solver(method, limit)
        result = solver.solve()
        print(result)
    except ValidationError as e:
        print(e)
        
    except Exception as e:
        print(e)
    

Enter method of solving problem [brute/formula] (formula):

Enter limit of numbers:

233168
